<a href="https://www.kaggle.com/code/manishkr1754/spam-mail-prediction?scriptVersionId=144421269" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

---
<center><h1>Spam Mail Prediction</h1></center>
<center><h3>Part of 30 Days 30 ML Projects Challenge</h3></center>

---

## 1) Understanding Problem Statement
---

In the age of digital communication, spam emails have become a significant nuisance, cluttering inboxes and potentially causing harm. This project addresses the issue of spam email detection through the lens of machine learning and data science.

The problem at hand falls into the category of **Binary Classification Machine Learning Problem**. The central objective is **to create a predictive model capable of distinguishing between spam and legitimate emails**. This model will rely on various email attributes and content features to make informed predictions, ultimately helping users filter out unwanted and potentially harmful messages. Moreover, It also involves use of **Natural Language Processing (NLP)** techniques for handling textual data.

## 2) Understanding Data
---

The project uses **Mail Data** which contains several variables (independent variables) and the outcome variable or dependent variable.

## 3) Getting System Ready
---
Importing required libraries


In [ ]:
import numpy as np
import pandas as pd

# for text data preprocessing
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# for model buidling
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

### Downloading stop words for text preprocessing

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

## 4) Data Eyeballing
---

### Laoding Data

In [ ]:
mail_data = pd.read_csv('Datasets/Day17_Mail_Data.csv') 

In [ ]:
mail_data

In [ ]:
print('The size of Dataframe is: ', mail_data.shape)
print('-'*100)
print('The Column Name, Record Count and Data Types are as follows: ')
mail_data.info()
print('-'*100)

In [ ]:
# Defining numerical & categorical columns
numeric_features = [feature for feature in mail_data.columns if mail_data[feature].dtype != 'O']
categorical_features = [feature for feature in mail_data.columns if mail_data[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

In [ ]:
print('Missing Value Presence in different columns of DataFrame are as follows : ')
print('-'*100)
total=mail_data.isnull().sum().sort_values(ascending=False)
percent=(mail_data.isnull().sum()/mail_data.isnull().count()*100).sort_values(ascending=False)
pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
print('Summary Statistics of numerical features for DataFrame are as follows:')
print('-'*100)
mail_data.describe(include='object')

## 5) Data Cleaning and Preprocessing
---

### Replace the null values with a null string (Only for demonstration, here it is not neede)

In [ ]:
mail_data = mail_data.where((pd.notnull(mail_data)),'')

In [ ]:
mail_data

### Label Encoding

#### label spam mail as 0;  ham mail as 1;

In [ ]:
mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

In [ ]:
mail_data

In [ ]:
mail_data['Category'].value_counts()

### Stemming

In [ ]:
porter_stemmer = PorterStemmer()

In [ ]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [porter_stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [ ]:
mail_data['Message'] = mail_data['Message'].apply(stemming)

In [ ]:
mail_data['Message']

## 6) Model Building
---

### Creating Feature Matrix (Independent Variables) & Target Variable (Dependent Variable)

In [ ]:
# separating the data and labels
X = mail_data['Message'] # Feature matrix
y = mail_data['Category'] # Target variable

In [ ]:
X

In [ ]:
y

In [ ]:
# convert Y_train and Y_test values as integers
y = y.astype('int')

### Feature Extraction

#### Transform the text data to feature vectors that can be used as input to the Logistic regression

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
X

In [ ]:
print(X)

### Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=45)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
print(y.shape, y_train.shape, y_test.shape)

### Model Comparison : Training & Evaluation

In [ ]:
models = [LogisticRegression, SVC, DecisionTreeClassifier, RandomForestClassifier]
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []

for model in models:
    classifier = model().fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    accuracy_scores.append(accuracy_score(y_test, y_pred))
    precision_scores.append(precision_score(y_test, y_pred))
    recall_scores.append(recall_score(y_test, y_pred))
    f1_scores.append(f1_score(y_test, y_pred))

In [ ]:
classification_metrics_df = pd.DataFrame({
    "Model": ["Logistic Regression", "SVM", "Decision Tree", "Random Forest"],
    "Accuracy": accuracy_scores,
    "Precision": precision_scores,
    "Recall": recall_scores,
    "F1 Score": f1_scores
})

classification_metrics_df.set_index('Model', inplace=True)
classification_metrics_df

### Inference

In the context of spam mail prediction, the performance metrics of four different models—Logistic Regression, SVM, Decision Tree, and Random Forest—have been evaluated. 

- Overall, all models exhibit high accuracy, with SVM and Random Forest leading the way, achieving accuracy scores of approximately 97.22% and 97.49%, respectively. This suggests that these models are effective at correctly classifying emails as spam or legitimate.

- When considering precision, recall, and F1 Score, the SVM and Random Forest models again excel, with precision scores exceeding 96.89% and recall scores reaching 100%, indicating minimal false positives and false negatives. Decision Tree also performs well in terms of precision and recall.

In summary, SVM and Random Forest models stand out as strong contenders for spam mail prediction, with high accuracy, precision, and recall scores. However, the choice of the best model may also depend on specific requirements such as computational efficiency and interpretability.